 # IIITD TechTree - Add Sheet Link to CSV

 This script performs the following:
 1. Reads course codes from `./techtree_table.csv`.
 2. For each row, identifies the first valid course code.
 3. Fetches the corresponding detail page URL (`viewDescription`).
 4. Parses the detail page HTML to find the `<iframe>` tag.
 5. Extracts the `src` attribute (the Google Sheet link) from the iframe.
 6. Adds this extracted link as a new column ("Sheet Link") to the original data.
 7. Saves the augmented data to a new CSV file (`techtree_table_with_links.csv`).
 8. Includes a delay between processing each row.


In [ ]:
# --- Setup: Import Libraries ---
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
from tqdm import tqdm # Use standard tqdm
import re
import warnings
from urllib.parse import urljoin

# Ignore specific warnings
warnings.filterwarnings('ignore', message='.*html5lib.*', module='bs4')
warnings.filterwarnings("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None # default='warn'

# --- Configuration ---
INPUT_CSV_FILE = "techtree_table.csv"         # CSV file containing the main course list
COURSE_CODE_COLUMN = "Course Code"           # Column name for course codes in the CSV
OUTPUT_CSV_FILE = "techtree_table_with_links.csv" # Output file name
BASE_URL = "https://techtree.iiitd.edu.in/" # Base URL for constructing links
DETAIL_URL_TEMPLATE = "https://techtree.iiitd.edu.in/viewDescription/filename?={}"

# --- ADJUSTABLE PARAMETERS ---
REQUEST_DELAY = 0.7  # Delay in seconds *between* fetching each detail page
REQUEST_TIMEOUT = 30 # Timeout for fetching detail pages

# --- Common Headers ---
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- Regex for Valid Course Code Format ---
VALID_CODE_PATTERN = re.compile(r"^[A-Za-z]{2,4}\d{3,4}[A-Za-z]*$")

# --- Helper Function to Get Sheet Link ---

# (Keep all previous imports and configuration unchanged until the get_sheet_link function)

# --- Modified Helper Function to Get Sheet Link ---
def get_sheet_link(course_code):
    """
    Improved version with better URL validation and redirect handling
    """
    if not course_code or not isinstance(course_code, str):
        return None

    detail_url = DETAIL_URL_TEMPLATE.format(course_code.strip())
    
    try:
        # Follow redirects and get final URL
        response = requests.get(
            detail_url,
            timeout=REQUEST_TIMEOUT,
            headers=HEADERS,
            allow_redirects=True
        )
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # 1. First try to find direct Google Sheet links in iframes
        iframes = soup.find_all('iframe')
        for iframe in iframes:
            if 'src' in iframe.attrs:
                sheet_url = urljoin(BASE_URL, iframe['src'])
                if 'docs.google.com/spreadsheets' in sheet_url:
                    return sheet_url

        # 2. Fallback: Look for embedded script data
        scripts = soup.find_all('script')
        for script in scripts:
            if script.string and 'google.com/spreadsheets' in script.string:
                match = re.search(r'https://docs\.google\.com/spreadsheets/d/e/[^\s"\']+', script.string)
                if match:
                    return match.group(0)

        # 3. Check for meta refresh redirects
        meta_refresh = soup.find('meta', attrs={'http-equiv': 'refresh'})
        if meta_refresh:
            content = meta_refresh.get('content', '')
            match = re.search(r'url=(.*)', content, re.IGNORECASE)
            if match:
                redirect_url = urljoin(BASE_URL, match.group(1))
                if 'google.com/spreadsheets' in redirect_url:
                    return redirect_url

        return None

    except requests.exceptions.RequestException as e:
        if hasattr(e, 'response') and e.response is not None:
            status = e.response.status_code
            if status == 404:
                return None
        return None
    except Exception as e:
        print(f"Error processing {course_code}: {str(e)}")
        return None


# --- Main Execution ---
print("--- IIITD TechTree - Add Sheet Link Script Started ---")

# 1. Read Input CSV
print(f"\n--- Reading Course Codes from {INPUT_CSV_FILE} ---")
try:
    main_df = pd.read_csv(INPUT_CSV_FILE)
    if COURSE_CODE_COLUMN not in main_df.columns:
        raise ValueError(f"Column '{COURSE_CODE_COLUMN}' not found. Available: {main_df.columns.tolist()}")
    print(f"Read {len(main_df)} rows from {INPUT_CSV_FILE}.")
except FileNotFoundError:
    print(f"[Critical Error] Input CSV file not found: {INPUT_CSV_FILE}")
    main_df = None
except Exception as e:
    print(f"[Critical Error] Failed to read input CSV {INPUT_CSV_FILE}: {e}")
    main_df = None

# 2. Extract Sheet Links
sheet_links = []
processed_rows = 0
codes_not_found = []

if main_df is not None:
    print("\n--- Fetching Detail Pages and Extracting Sheet Links ---")
    # Iterate row by row using tqdm for progress
    for index, row in tqdm(main_df.iterrows(), total=main_df.shape[0], desc="Processing Rows"):
        processed_rows += 1
        code_entry = row[COURSE_CODE_COLUMN]
        code_entry_str = str(code_entry).strip() if pd.notna(code_entry) else ""

        first_valid_code = None
        link_found = None

        if code_entry_str:
            potential_codes = [c.strip() for c in code_entry_str.split('/') if c.strip()]
            # Find the first code in the list that matches the valid pattern
            for code in potential_codes:
                if VALID_CODE_PATTERN.match(code):
                    first_valid_code = code
                    break # Use the first valid one we find

        if first_valid_code:
            link_found = get_sheet_link(first_valid_code)
            if link_found is None:
                 codes_not_found.append(first_valid_code) # Log code if link extraction failed

        # Append the found link (or None if no valid code or link not found)
        sheet_links.append(link_found if link_found else '') # Append empty string for consistency in CSV

        # --- Delay ---
        time.sleep(REQUEST_DELAY)

    # Sanity check
    if len(sheet_links) != len(main_df):
        print(f"[Warning] Mismatch in link count ({len(sheet_links)}) and DataFrame rows ({len(main_df)}).")
        # Pad sheet_links if necessary (though this shouldn't happen with the current loop)
        sheet_links.extend([''] * (len(main_df) - len(sheet_links)))

    # 3. Add Column to DataFrame
    try:
        main_df['Sheet Link'] = sheet_links
        print("\nAdded 'Sheet Link' column to DataFrame.")
    except Exception as e:
        print(f"[Error] Failed to add 'Sheet Link' column: {e}")
        main_df = None # Prevent saving potentially corrupted data

    # 4. Save Output CSV
    if main_df is not None:
        print(f"\n--- Saving augmented data to {OUTPUT_CSV_FILE} ---")
        try:
            main_df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8')
            print(f"Successfully saved data to {os.path.abspath(OUTPUT_CSV_FILE)}")
            print("\n--- Sample of Augmented Data ---")
            print(main_df[['Course Code', 'Sheet Link']].head())

            if codes_not_found:
                 print(f"\n--- Summary ---")
                 print(f"Could not find a valid Google Sheet link for {len(codes_not_found)} codes (using first valid code per row):")
                 print(", ".join(sorted(list(set(codes_not_found)))))

        except Exception as e:
            print(f"[Error] Failed to save output CSV file: {e}")

else:
    print("\nScript halted due to errors reading the input CSV.")

print("\n--- Script Finished ---")

--- IIITD TechTree - Add Sheet Link Script Started ---

--- Reading Course Codes from techtree_table.csv ---
Read 425 rows from techtree_table.csv.

--- Fetching Detail Pages and Extracting Sheet Links ---


Processing Rows: 100%|██████████| 425/425 [17:07<00:00,  2.42s/it]   


Added 'Sheet Link' column to DataFrame.

--- Saving augmented data to techtree_table_with_links.csv ---
Successfully saved data to c:\Users\amart\OneDrive\Desktop\Code\Askalma\techtree_table_with_links.csv

--- Sample of Augmented Data ---
  Course Code                                         Sheet Link
0      BIO101  https://docs.google.com/spreadsheets/d/e/2PACX...
1      BIO211  https://docs.google.com/spreadsheets/d/1N1fhCw...
2      BIO213  https://docs.google.com/spreadsheets/d/e/2PACX...
3      BIO214  https://docs.google.com/spreadsheets/d/e/2PACX...
4      BIO221  https://docs.google.com/spreadsheets/d/e/2PACX...

--- Summary ---
Could not find a valid Google Sheet link for 4 codes (using first valid code per row):
ECE240, MTH506, MTH562, SSH326

--- Script Finished ---


 # IIITD TechTree - Parse Sheet Links via Selenium & Manual DIV Parsing

 This script performs the following:
 1. Reads `./techtree_table_with_links.csv`.
 2. Filters rows based ONLY on having a valid starting "Sheet Link" URL.
 3. Uses `N_WORKERS` concurrent workers, each managing a Selenium WebDriver instance.
 4. For each valid Sheet Link URL:
     a. Uses Selenium to navigate to the URL and wait for grid content to render.
     b. Gets the rendered page source.
     c. **Uses a manual BeautifulSoup parser to extract data from the `div`-based grid structure.**
     d. Saves the extracted data as a DataFrame to "Sheet1" in an **individual Excel file** named `{course_code}.xlsx` in the `./Techtree/` directory.
 5. Tracks and reports successful extractions and specific failures accurately based on individual codes.


In [ ]:
# --- Setup: Import Libraries ---
import pandas as pd
import os
import time
from tqdm import tqdm
import re
import warnings
from io import StringIO
import concurrent.futures
import openpyxl # Required for saving Excel files
import threading

# Selenium Imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, TimeoutException
from selenium.webdriver.chrome.service import Service as ChromeService

# WebDriver Manager (recommended)
try:
    from webdriver_manager.chrome import ChromeDriverManager
    USE_WEBDRIVER_MANAGER = True
except ImportError:
    print("[Info] webdriver-manager not found. Ensure chromedriver is in PATH or set CHROMEDRIVER_PATH.")
    USE_WEBDRIVER_MANAGER = False
    CHROMEDRIVER_PATH = None # e.g., r"C:\path\to\chromedriver.exe"

from bs4 import BeautifulSoup, Tag

# Ignore specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None # default='warn'

# --- Configuration ---
INPUT_CSV_FILE = "techtree_table_with_links.csv" # Input CSV with sheet links
COURSE_CODE_COLUMN = "Course Code"           # Column for naming output files
SHEET_LINK_COLUMN = "Sheet Link"             # Column containing the Google Sheet URLs
OUTPUT_DIR = "Techtree"                      # Output directory for Excel files

# --- ADJUSTABLE PARAMETERS ---
N_WORKERS = 5   # Reduce workers initially for stability with Selenium
REQUEST_DELAY_PER_WORKER = 0.2 # Can increase delay if needed
PAGE_LOAD_TIMEOUT = 60 # Increased timeout for Selenium page load
WAIT_FOR_ELEMENT_TIMEOUT = 20 # Max seconds to wait for grid elements

# --- Common Headers for Requests (less critical now, but good practice) ---
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- Thread-safe storage for results ---
processed_lock = threading.Lock()
successful_codes_set = set()
failed_codes_dict = {} # Maps code -> error message

# --- Helper: Manual BS4 Parsing for Rendered Google Sheet ---
def manual_parse_rendered_sheet(soup):
    """
    Parses the RENDERED HTML (obtained via Selenium) of a Google Sheet
    using BeautifulSoup to extract data from div structures.
    Returns a pandas DataFrame if successful, else None.
    """
    data_rows = []
    headers = []
    header_row_index = -1

    # Find the main grid container - adjust selector if needed based on inspection
    grid_container = soup.find('div', {'id': 'waffle-grid-container'}) # Common ID
    if not grid_container:
        grid_container = soup.find('div', class_='ritzgrid') # Another common class
    if not grid_container:
        grid_container = soup.body # Fallback

    # Find potential rows (often <tr> in the rendered structure, even if complex)
    rows = grid_container.find_all('tr')
    if not rows: # Fallback to divs if no <tr>
         # This selector might need refinement based on inspecting actual rendered HTML
         rows = grid_container.find_all(lambda tag: tag.name == 'div' and 'grid-row' in tag.get('class', []))

    for i, row_tag in enumerate(rows):
        # Find potential cells within the row
        cells = row_tag.find_all(['td', 'th', lambda tag: tag.name == 'div' and tag.get('role') == 'gridcell'])
        # Extract text, preferring inner text if cells are complex
        row_data = [cell.get_text(separator=' ', strip=True) for cell in cells]

        if not any(row_data): continue # Skip fully empty rows

        # Heuristic Header Detection: Often the first few non-empty rows might contain headers or titles.
        # Let's assume the first row with significant content acts as the primary header row.
        # Or look for <th> tags if present.
        is_header = any(cell.name == 'th' for cell in cells) or (header_row_index == -1 and len(row_data) > 1) # Simple: first row is header

        if is_header and header_row_index == -1:
             headers = row_data
             header_row_index = i
        elif header_row_index != -1: # Collect data rows after header identified
             # Basic alignment with header length
             if headers:
                  diff = len(headers) - len(row_data)
                  if diff > 0: row_data.extend([''] * diff)
                  elif diff < 0: row_data = row_data[:len(headers)]
             data_rows.append(row_data)

    if data_rows:
        df = pd.DataFrame(data_rows, columns=headers if headers else None)
        df.dropna(axis=1, how='all', inplace=True) # Drop fully empty columns
        df.dropna(axis=0, how='all', inplace=True) # Drop fully empty rows
        return df # Return the single DataFrame
    else:
        # print("    [Manual Parse Debug] No data rows extracted.")
        return None # Return None if parsing yields no data


# --- Helper Function to Initialize WebDriver ---
def initialize_driver():
    # print("Initializing WebDriver...") # Reduce noise
    options = webdriver.ChromeOptions()
    options.add_argument("--headless") # Run headless for automation
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--allow-running-insecure-content')
    options.add_argument(f'user-agent={HEADERS["User-Agent"]}') # Set user agent
    options.add_argument("--log-level=3")

    driver = None
    try:
        if USE_WEBDRIVER_MANAGER:
            service = ChromeService(ChromeDriverManager().install())
            driver = webdriver.Chrome(service=service, options=options)
        elif CHROMEDRIVER_PATH and os.path.exists(CHROMEDRIVER_PATH):
            service = ChromeService(executable_path=CHROMEDRIVER_PATH)
            driver = webdriver.Chrome(service=service, options=options)
        else:
           raise ValueError("WebDriver config error.")
        driver.set_page_load_timeout(PAGE_LOAD_TIMEOUT)
        # print("WebDriver initialized.") # Reduce noise
        return driver
    except Exception as e:
        print(f"[Error] WebDriver Initialization Failed: {e}")
        return None

# --- Helper Function to Process a Single Sheet Link using SELENIUM ---

# Thread-local storage for WebDriver instances
thread_local = threading.local()

def get_driver():
    """Gets or creates a WebDriver instance for the current thread."""
    if not hasattr(thread_local, "driver") or thread_local.driver is None:
        thread_local.driver = initialize_driver()
    return thread_local.driver

def close_driver():
    """Closes the WebDriver instance for the current thread if it exists."""
    if hasattr(thread_local, "driver") and thread_local.driver is not None:
        # print(f"Closing driver for thread {threading.current_thread().name}") # Debug
        thread_local.driver.quit()
        thread_local.driver = None

def process_link_selenium(course_code, sheet_link):
    """
    Uses Selenium to load sheet link, waits for grid, manually parses divs, saves to Excel.
    Returns course_code on success, raises Exception on failure.
    """
    safe_course_code = re.sub(r'[\\/*?:"<>|]',"", str(course_code)).strip()
    if not safe_course_code: raise ValueError("Invalid course code for filename")
    if not isinstance(sheet_link, str) or not sheet_link.startswith('http'): raise ValueError("Invalid Sheet Link URL")

    course_file_path = os.path.join(OUTPUT_DIR, f"{safe_course_code}.xlsx")
    parsed_df = None

    driver = get_driver() # Get driver specific to this thread
    if driver is None:
        raise RuntimeError("WebDriver not available for this thread")

    # 1. Navigate and Wait
    try:
        # print(f"  Navigating to {sheet_link} for {course_code}") # Debug
        driver.get(sheet_link)
        # Wait for an element that indicates the grid data is loaded
        # Adjust selector based on actual inspection of a loaded sheet's HTML
        wait_selector = (By.CSS_SELECTOR, "div.grid-container div[role='gridcell'], div.ritzgrid-content div[role='gridcell']") # Look for grid cells
        WebDriverWait(driver, WAIT_FOR_ELEMENT_TIMEOUT).until(
            EC.presence_of_element_located(wait_selector)
        )
        # print(f"  Content detected for {course_code}") # Debug
        # Optional short pause after element found, sometimes helps ensure full render
        time.sleep(1)

    except TimeoutException:
        raise TimeoutError(f"Timeout waiting for grid elements on {sheet_link}")
    except WebDriverException as e:
        raise RuntimeError(f"WebDriver error during navigation/wait for {sheet_link}: {e}") from e
    except Exception as e:
        raise RuntimeError(f"Unexpected error during navigation/wait: {e}") from e


    # 2. Get Rendered Source and Parse Manually
    try:
        rendered_html = driver.page_source
        if not rendered_html: raise ValueError("Selenium returned empty page source")

        soup = BeautifulSoup(rendered_html, 'html.parser')
        parsed_df = manual_parse_rendered_sheet(soup) # Use the manual parser

        if parsed_df is None:
             raise LookupError("Manual BS4 parsing did not extract a DataFrame from rendered HTML")

    except Exception as e:
         raise RuntimeError(f"Error parsing rendered HTML for {course_code}: {e}") from e


    # 3. Save Extracted DataFrame to Excel
    try:
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        with pd.ExcelWriter(course_file_path, engine='openpyxl') as writer:
             # Manual parse returns one DF
             df_str = parsed_df.fillna('').astype(str)
             df_str.to_excel(writer, sheet_name="Sheet1", index=False)

        if not os.path.exists(course_file_path) or os.path.getsize(course_file_path) < 50:
             if os.path.exists(course_file_path): os.remove(course_file_path)
             raise LookupError("Manual parsing resulted in an empty Excel file.")

        return course_code # Success

    except Exception as save_err:
        raise IOError(f"Error saving Excel file {course_file_path}: {save_err}") from save_err


# --- Main Execution ---
print("--- IIITD TechTree - Parse Sheet Links via Selenium & Manual DIV Parsing ---")

# 1. Create Output Directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Using output directory: {OUTPUT_DIR}")

# 2. Read Input CSV and Prepare Tasks
print(f"\n--- Reading Links from {INPUT_CSV_FILE} ---")
tasks_to_process = []
skipped_rows_info = []
all_attempted_codes = set()

try:
    main_df = pd.read_csv(INPUT_CSV_FILE)
    print(f"Read {len(main_df)} total rows from CSV.")
    if COURSE_CODE_COLUMN not in main_df.columns or SHEET_LINK_COLUMN not in main_df.columns:
        raise ValueError(f"Required columns missing. Need '{COURSE_CODE_COLUMN}', '{SHEET_LINK_COLUMN}'. Found: {main_df.columns.tolist()}")

    for index, row in main_df.iterrows():
        code_val = row[COURSE_CODE_COLUMN]
        link_val = row[SHEET_LINK_COLUMN]
        code_str = str(code_val).strip() if pd.notna(code_val) else ""
        link_str = str(link_val).strip() if pd.notna(link_val) else ""

        if link_str.startswith('http') and code_str:
            tasks_to_process.append((code_str, link_str))
            all_attempted_codes.add(code_str)
        else:
            reason = "Missing/invalid link" if not link_str.startswith('http') else "Missing Course Code"
            skipped_rows_info.append((index + 2, code_str, link_str, reason))

    print(f"Prepared {len(tasks_to_process)} tasks with valid codes and sheet links.")
    if skipped_rows_info: print(f"Skipped {len(skipped_rows_info)} rows (details at end).")

except FileNotFoundError: print(f"[Critical Error] Input CSV file not found: {INPUT_CSV_FILE}"); tasks_to_process = []
except Exception as e: print(f"[Critical Error] Failed to read/process CSV: {e}"); tasks_to_process = []

# 3. Process Links Concurrently using Selenium
if tasks_to_process:
    print(f"\n--- Fetching and Parsing {len(tasks_to_process)} Sheet Links using {N_WORKERS} workers (Selenium) ---")
    successful_codes_set.clear(); failed_codes_dict.clear()

    # Use ThreadPoolExecutor. Each thread gets its own WebDriver instance via thread_local.
    with concurrent.futures.ThreadPoolExecutor(max_workers=N_WORKERS, initializer=get_driver, initargs=()) as executor:
        future_to_task = {executor.submit(process_link_selenium, code, link): (code, link) for code, link in tasks_to_process}

        for future in tqdm(concurrent.futures.as_completed(future_to_task), total=len(tasks_to_process), desc="Processing Links"):
            code, link = future_to_task[future]
            try:
                result_code = future.result() # course_code on success
                with processed_lock:
                     successful_codes_set.add(result_code)
                     if result_code in failed_codes_dict: del failed_codes_dict[result_code]
            except Exception as exc:
                 error_message = f"{type(exc).__name__}: {exc}"
                 with processed_lock:
                      if code not in successful_codes_set: failed_codes_dict[code] = error_message
            # No explicit delay here, let thread pool manage based on task completion

    # Ensure all thread-local drivers are closed after pool finishes
    # This requires running close_driver in the context where the threads were managed.
    # For simplicity here, assume threads exit cleanly. In complex scenarios, may need explicit cleanup loop.
    print("Closing any remaining WebDriver instances (best effort)...")
    # Note: This simple approach might miss drivers if threads error out unexpectedly.
    # A more robust cleanup would involve tracking driver instances.
    # Since we re-initialize per thread if needed, this is less critical.


    # 4. Final Reporting
    print(f"\n--- Processing Complete ---")
    total_successful_codes = len(successful_codes_set)
    print(f"Successfully generated Excel files for: {total_successful_codes} course codes.")

    final_failed_codes = all_attempted_codes - successful_codes_set
    if final_failed_codes:
        print(f"Failed to process/save details for: {len(final_failed_codes)} course codes:")
        sorted_failed_codes = sorted(list(final_failed_codes))
        for code in sorted_failed_codes:
            error_msg = failed_codes_dict.get(code, "Failure reason not recorded.")
            print(f"  - {code}: {error_msg}")
    else:
        if tasks_to_process: print("All attempted course codes were processed successfully!")

    if skipped_rows_info:
        print("\n--- Rows Skipped During Initial CSV Reading ---")
        for row_num, code_val, link_val, reason in skipped_rows_info:
            print(f"  - Row {row_num}: Code='{code_val}', Link='{link_val}' ({reason})")

else:
    print("\nNo valid tasks (course code + sheet link) found to process.")
    if skipped_rows_info: # Still report skipped rows
         print("\n--- Rows Skipped During Initial CSV Reading ---")
         for row_num, code_val, link_val, reason in skipped_rows_info:
             print(f"  - Row {row_num}: Code='{code_val}', Link='{link_val}' ({reason})")

print("\n--- Script Finished ---")

--- IIITD TechTree - Parse Sheet Links via Selenium & Manual DIV Parsing ---
Using output directory: Techtree

--- Reading Links from techtree_table_with_links.csv ---
Read 425 total rows from CSV.
Prepared 414 tasks with valid codes and sheet links.
Skipped 11 rows (details at end).

--- Fetching and Parsing 414 Sheet Links using 5 workers (Selenium) ---


Processing Links: 100%|██████████| 414/414 [29:29<00:00,  4.27s/it]

Closing any remaining WebDriver instances (best effort)...

--- Processing Complete ---
Successfully generated Excel files for: 0 course codes.
Failed to process/save details for: 410 course codes:
  - BIO101: TimeoutError: Timeout waiting for grid elements on https://docs.google.com/spreadsheets/d/e/2PACX-1vQZyQdcTzRSTTdex_nZyO2XIHGYQsNi5quu2-4SMngwINszA4XB-ZJXlwF_ETdpaBgSACl2l3EKZcMd/pubhtml?widget=true&amp;headers=false
  - BIO102: TimeoutError: Timeout waiting for grid elements on https://docs.google.com/spreadsheets/d/e/2PACX-1vQ_1As-6k1beEfnbkwc0nESgj-Ktn0a18Qm_mJ7O1x0NingoTaHE34u3AvVaO4RCQ/pubhtml?widget=true&amp;headers=false
  - BIO211: TimeoutError: Timeout waiting for grid elements on https://docs.google.com/spreadsheets/d/1N1fhCwaaBxHzdS2km-KrEUg4lZzlwKZN/pubhtml?widget=true&amp;headers=false
  - BIO213: TimeoutError: Timeout waiting for grid elements on https://docs.google.com/spreadsheets/d/e/2PACX-1vTNK82nmg34cFhO9WX5ZR7dCmZVPg_wU1q5YTzQi-S-tebvkhipxVzWi5h9cb1BD8nz3zcPC3

Some manual testing to verify the the structure, for llm calling and other forms of parsing. 😊

In [2]:
import pandas as pd

# Load the Excel file
file_path = "./Techtree/BIO534.xlsx"  # Replace with the actual file path
df = pd.read_excel(file_path)
print(df)


    Unnamed: 0                                         Unnamed: 1  \
0            1                                        Course Code   
1            2                                        Course Name   
2            3                                            Credits   
3            4                                  Course Offered to   
4            5                                 Course Description   
5            6                                     Pre-requisites   
6            7                          Pre-requisite (Mandatory)   
7            8                                                NaN   
8            9               *Please insert more rows if required   
9           10                                    Course Outcomes   
10          11                                                CO1   
11          12  Explain and classify different properties of n...   
12          13                                Weekly Lecture Plan   
13          14                    


# Excel-to-JSON Converter using Local LLM

 This script processes .xlsx files from a specified folder,
 extracts their contents, sends them to a local LLM for JSON 
 structuring, and saves the outputs as .json files.

 ## Configuration:
   - INPUT_FOLDER_PATH: Directory with Excel files
   - OUTPUT_JSON_FOLDER: Destination for JSON files
   - LLM_API_URL: Local LLM endpoint
   - LLM_MODEL_NAME: Model to use (e.g., deepseek-r1-distill-qwen-7b)

 ## Process:
   1. Read and flatten Excel files into text
   2. Format a structured prompt with JSON schema
   3. Send to LLM and receive raw JSON output
   4. Extract and validate JSON from response
   5. Save valid JSON, log any errors

 ## Output JSON includes:
   - Course Code, Name, Credits, Description
   - Prerequisites and Outcomes
   - Weekly Lecture Plan (list of weeks, COs met, etc.)
   - Assessment Plan and Resource Material



In [1]:
import os
import glob
import pandas as pd
import requests
import json
import re # For more robust extraction

# --- Configuration ---
INPUT_FOLDER_PATH = "./Techtree"  # Folder containing your .xlsx files
OUTPUT_JSON_FOLDER = "./jsons"
LLM_API_URL = "http://localhost:1234/v1/chat/completions"
LLM_MODEL_NAME = "deepseek-r1-distill-qwen-7b" # Or your specific model

# The specific prompt for the LLM
# Note the {{ and }} are escaped as {{ and }} in f-strings, but here it's a raw string.
# The desired JSON format part is already correctly formatted for a string.
LLM_SYSTEM_PROMPT = """You are an AI assistant. Your task is to extract information from the provided text and structure it into a specific JSON format.
You MUST ONLY output the raw JSON object. Do not include any introductory text, explanations, or markdown formatting like ```json ... ```.
The output must be directly parsable as JSON.
Handle missing values gracefully (e.g., empty strings "" or null).
Ensure "Credits" is a number if present and valid, otherwise a string.
"COs Met" should be a list of strings.
"Weekly Lecture Plan" should be a list of objects, where each object represents a week's plan.
"Assessment Plan" values should ideally be numbers (e.g., percentages), or strings if not numeric.
"""

LLM_USER_PROMPT_TEMPLATE = """From the given file content below, provide the information in the desired JSON format:
{{
  "Course Code": "...",
  "Course Name": "...",
  "Credits": "...",
  "Course Offered to": "...",
  "Course Description": "...",
  "Prerequisites": {{
    "Mandatory": "...",
    "Desirable": "...",
    "Other": "..."
  }},
  "Course Outcomes": {{
    "CO1": "...",
    "CO2": "...",
    "CO3": "...",
    "CO4": "..."
  }},
  "Weekly Lecture Plan": [
    {{
      "Week": "...",
      "Lecture Topic": "...",
      "COs Met": ["CO1", "CO2"],
      "Assignments": "..."
    }}
  ],
  "Assessment Plan": {{
    "Quiz exams": "...",
    "Midsem": "...",
    "HW assignments": "...",
    "Endsem": "..."
  }},
  "Resource Material": {{
    "Textbook": "...",
    "Reference Book": "..."
  }}
}}

--- FILE CONTENT BEGIN ---
{file_text_content}
--- FILE CONTENT END ---
"""

# --- Helper Functions ---

def excel_to_text(xlsx_filepath):
    """
    Converts all sheets of an Excel file to a single text string.
    Each sheet's content is appended, prefixed by its name.
    """
    try:
        xls = pd.ExcelFile(xlsx_filepath)
        text_parts = []
        if not xls.sheet_names:
            print(f"    Warning: No sheets found in {xlsx_filepath}.")
            return None

        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name, header=None) # Read without auto-header detection
            # Convert DataFrame to a string, trying to get all cell values as text.
            sheet_text_content = []
            for _, row in df.iterrows():
                # Join non-NaN cells in the row, converting them to string
                row_values = [str(cell) for cell in row if pd.notna(cell)]
                if row_values: # only add rows that have some content
                    sheet_text_content.append(" | ".join(row_values)) # Simple separator
            
            if sheet_text_content:
                text_parts.append(f"--- Sheet: {sheet_name} ---\n" + "\n".join(sheet_text_content))
        
        full_text = "\n\n".join(text_parts)
        return full_text if full_text.strip() else None
    except Exception as e:
        print(f"    Error converting Excel '{xlsx_filepath}' to text: {e}")
        return None

def call_llm(text_content):
    """
    Sends the text content to the local LLM and returns the response content.
    """
    headers = {
        "Content-Type": "application/json"
    }
    user_prompt = LLM_USER_PROMPT_TEMPLATE.format(file_text_content=text_content)
    payload = {
        "model": LLM_MODEL_NAME,
        "messages": [
            {"role": "system", "content": LLM_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": 0.2, # Lower temperature for more deterministic JSON output
        "max_tokens": 4096, # Adjust if needed, ensure it's enough for the JSON
        "stream": False
    }
    try:
        print("    Sending request to LLM...")
        response = requests.post(LLM_API_URL, headers=headers, json=payload, timeout=120) # 2 min timeout
        response.raise_for_status()  # Raises an HTTPError for bad responses (4XX or 5XX)
        
        response_data = response.json()
        if response_data.get("choices") and len(response_data["choices"]) > 0:
            return response_data["choices"][0]["message"]["content"]
        else:
            print(f"    Error: LLM response does not contain 'choices': {response_data}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"    Error calling LLM API: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"    Error decoding LLM JSON response: {e}")
        print(f"    Raw response: {response.text[:500]}...") # Print part of raw response
        return None


def extract_json_from_llm_output(llm_raw_output):
    """
    Extracts the JSON string after the '</think>' tag.
    Tries to find the first '{' after the tag for more robustness.
    """
    if not llm_raw_output:
        return None

    think_tag = "</think>"
    think_tag_pos = llm_raw_output.rfind(think_tag) # Use rfind to get the last occurrence

    if think_tag_pos != -1:
        content_after_think = llm_raw_output[think_tag_pos + len(think_tag):]
    else:
        # If no </think> tag, assume the whole output might be the JSON or needs cleanup
        print("    Warning: '</think>' tag not found. Attempting to parse from start of content.")
        content_after_think = llm_raw_output

    # Try to find the start of the JSON object '{' or array '['
    json_start_brace = content_after_think.find('{')
    json_start_bracket = content_after_think.find('[')

    start_pos = -1

    if json_start_brace != -1 and json_start_bracket != -1:
        start_pos = min(json_start_brace, json_start_bracket)
    elif json_start_brace != -1:
        start_pos = json_start_brace
    elif json_start_bracket != -1:
        start_pos = json_start_bracket
    
    if start_pos != -1:
        potential_json_str = content_after_think[start_pos:]
        # Try to find matching closing brace/bracket for a more robust extraction
        try:
            # This is a bit naive for complex nested structures but works for simple cases.
            # A full JSON parser would be needed for perfect extraction from messy text.
            # Here, we assume the LLM is mostly compliant with "only JSON" after the tag.
            json.loads(potential_json_str) # Validate if it's a full JSON
            return potential_json_str.strip()
        except json.JSONDecodeError:
            # If it's not a full JSON, try to find the last '}' or ']'
            # This is heuristics and might not be perfect
            json_end_brace = potential_json_str.rfind('}')
            json_end_bracket = potential_json_str.rfind(']')
            end_pos = max(json_end_brace, json_end_bracket)
            if end_pos != -1:
                return potential_json_str[:end_pos+1].strip()
            else:
                print("    Warning: Could not determine valid JSON boundaries after stripping.")
                return potential_json_str.strip() # Return stripped anyway
    else:
        print("    Warning: No '{' or '[' found in content after '</think>' (or in whole content).")
        return content_after_think.strip() # Return stripped content, might be non-JSON

# --- Main Script ---
def main():
    if not os.path.exists(INPUT_FOLDER_PATH):
        print(f"Error: Input folder '{INPUT_FOLDER_PATH}' not found.")
        return
    if not os.path.exists(OUTPUT_JSON_FOLDER):
        os.makedirs(OUTPUT_JSON_FOLDER)
        print(f"Created output folder: '{OUTPUT_JSON_FOLDER}'")

    xlsx_files = glob.glob(os.path.join(INPUT_FOLDER_PATH, "*.xlsx"))
    if not xlsx_files:
        print(f"No .xlsx files found in '{INPUT_FOLDER_PATH}'.")
        return

    print(f"Found {len(xlsx_files)} .xlsx files to process.")

    for xlsx_filepath in xlsx_files:
        filename = os.path.basename(xlsx_filepath)
        print(f"\nProcessing file: {filename}")

        # 1. Convert Excel to Text
        print("  1. Converting Excel to text...")
        text_content = excel_to_text(xlsx_filepath)
        if not text_content:
            print(f"    Skipping {filename} due to conversion error or empty content.")
            continue
        # print(f"    Text content (first 200 chars): {text_content[:200]}...") # For debugging

        # 2. Call LLM with the text content
        print("  2. Calling LLM for JSON extraction...")
        llm_response_content = call_llm(text_content)
        if not llm_response_content:
            print(f"    Skipping {filename} due to LLM API error or empty response.")
            continue
        # print(f"    LLM raw response (first 200 chars): {llm_response_content[:200]}...") # For debugging

        # 3. Extract JSON part from LLM response
        print("  3. Extracting JSON from LLM response...")
        json_string = extract_json_from_llm_output(llm_response_content)
        if not json_string:
            print(f"    Could not extract a potential JSON string for {filename}. LLM output: {llm_response_content[:200]}")
            continue
        
        # 4. Validate and Save JSON
        print("  4. Validating and saving JSON...")
        try:
            # Validate that the extracted string is indeed valid JSON
            parsed_json = json.loads(json_string) 
            # Re-serialize to ensure it's clean and consistently formatted (optional, but good practice)
            # If you trust the LLM's formatting, you can just save json_string directly
            pretty_json_output = json.dumps(parsed_json, indent=2) 

            base_name_without_ext = os.path.splitext(filename)[0]
            output_json_path = os.path.join(OUTPUT_JSON_FOLDER, f"{base_name_without_ext}.json")
            
            with open(output_json_path, 'w', encoding='utf-8') as f:
                f.write(pretty_json_output) # Save the pretty printed version
            print(f"    Successfully saved JSON to: {output_json_path}")

        except json.JSONDecodeError as e:
            print(f"    Error: Extracted string for {filename} is not valid JSON: {e}")
            print(f"    Problematic string (first 300 chars): {json_string[:300]}...")
            # Optionally save the problematic string for debugging
            debug_filename = os.path.join(OUTPUT_JSON_FOLDER, f"{os.path.splitext(filename)[0]}_error.txt")
            with open(debug_filename, 'w', encoding='utf-8') as f_err:
                f_err.write(f"--- LLM RAW OUTPUT ---\n{llm_response_content}\n\n--- EXTRACTED STRING ---\n{json_string}")
            print(f"    Saved problematic output to {debug_filename}")
        except Exception as e:
            print(f"    An unexpected error occurred while saving JSON for {filename}: {e}")

    print("\nProcessing complete.")

if __name__ == "__main__":
    main()

Found 252 .xlsx files to process.

Processing file: BIO213.xlsx
  1. Converting Excel to text...
  2. Calling LLM for JSON extraction...
    Sending request to LLM...
  3. Extracting JSON from LLM response...
  4. Validating and saving JSON...
    Successfully saved JSON to: ./jsons\BIO213.json

Processing file: BIO214.xlsx
  1. Converting Excel to text...
  2. Calling LLM for JSON extraction...
    Sending request to LLM...
  3. Extracting JSON from LLM response...
  4. Validating and saving JSON...
    Successfully saved JSON to: ./jsons\BIO214.json

Processing file: BIO361.xlsx
  1. Converting Excel to text...
  2. Calling LLM for JSON extraction...
    Sending request to LLM...
  3. Extracting JSON from LLM response...
  4. Validating and saving JSON...
    Successfully saved JSON to: ./jsons\BIO361.json

Processing file: BIO512.xlsx
  1. Converting Excel to text...
  2. Calling LLM for JSON extraction...
    Sending request to LLM...
  3. Extracting JSON from LLM response...
  4. 